In [3]:
import os

import json
import tensorflow as tf
from flask import Flask, render_template, request, url_for, request, jsonify
import numpy as np


# Set TensorFlow to use CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


PATHWAY_DICT_PATH = 'my_data/precily_pathways_dict.npy'
GDSC_DICT_PATH = 'my_data/precily_labels_dict.npy'
SMILES_PATH = 'my_data/canonical_smiles.json'
MODEL_PATH_prc = 'tf2_model_all/precily_cv_5.hdf5'  # Update with the actual path
MODEL_PATH_tcnn = 'tf2_model_all/best_model.h5'  # Update with the actual path

In [4]:
pathway_dict = np.load(PATHWAY_DICT_PATH, allow_pickle=True).item()
print(f'Pathway Dict: {len(pathway_dict)}')
gdsc_dict = np.load(GDSC_DICT_PATH, allow_pickle=True).item()
print(f'GDSC Dict: {len(gdsc_dict)}')

#model taking final 600 pathways
model_prc = tf.keras.models.load_model(MODEL_PATH_prc)
# Load the model
model_tcnn = tf.keras.models.load_model(MODEL_PATH_tcnn)

with open(SMILES_PATH, 'r') as fp:
    canonical_smiles = json.load(fp)

Pathway Dict: 14256
GDSC Dict: 14256


2024-01-05 15:58:39.817523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-05 15:58:39.817550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: pc
2024-01-05 15:58:39.817555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: pc
2024-01-05 15:58:39.817602: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 470.223.2
2024-01-05 15:58:39.817619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.223.2
2024-01-05 15:58:39.817623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 470.223.2


In [5]:
def calculate_y(z):
    """
    Calculate y from z using the inverse of the formula: y = (1/z - 1)**(-10)
    
    Parameters:
    z (float): Input value for z
    
    Returns:
    float: Calculated value for y
    """
    return (1/z - 1)**(-10)
    
def calculate_smiles(drug_name):
    # load the SMILES json file
    global canonical_smiles
    canonical_smile = canonical_smiles[drug_name]
    return canonical_smile

def calculate_genetic_feature(cell_line_name):
    # load genetic feature dictionary
    cell_mut_dict = np.load("tcnn/my_data/cell_mut_matrix.npy", allow_pickle=True, encoding="latin1").item()
    cell_mut = cell_mut_dict["cell_mut"]
    cell_dict = cell_mut_dict['cell_dict']
    
    # calculate id-number of a cell
    cell_id = cell_dict[cell_line_name]
    cell_genetic_feature = cell_mut[cell_id]
    return cell_genetic_feature

def preprocess_input_tcnn(drug_name, cell_line_name):
    # Assuming drug_data and cell_data are single elements, convert them to numpy arrays
    drug_data = np.array([calculate_smiles(drug_name)])
    drug_data = np.transpose(drug_data, (0, 2, 1))  # Transpose the last two dimensions

    cell_data = np.array([calculate_genetic_feature(cell_line_name)])
    
    # print(f'DRUG Name: {drug_data}, CELL data: {cell_data}')

    # Make predictions using the loaded model
    predictions = model_tcnn.predict([drug_data, cell_data])

    # Return the predictions
    return predictions[0][0]

def preprocess_input_prc(drug_name, cell_line_name):
    global pathway_dict
    key = (str(cell_line_name), str(drug_name))
    
    # print(f'key: {key}')
    # print(f'Pathway Dict: {len(pathway_dict)}')
    
    # Assuming drug_data and cell_data are single elements, convert them to numpy arrays
    pathway_info = np.array(pathway_dict[key]).reshape(1,-1)
    #print(f'Pathway Info: {pathway_info}')
    # Make predictions using the loaded model
    predictions = model_prc.predict(pathway_info)
    # Return the predictions
    return predictions[0][0]


def preprocess_input_gdsc(drug_name, cell_line_name):
    # Make predictions using the loaded model
    global gdsc_dict
    key = (str(cell_line_name), str(drug_name))
    print(f'GDSC Dict: {gdsc_dict.keys()}')
    print(f'key: {key}')
    print(f'Status: {key in gdsc_dict}')
    predictions = gdsc_dict[key]
    return predictions

In [6]:

# instance of a flask object
app = Flask(__name__)
    
@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = None

    if request.method == 'POST':
        # Get input values from the form
        drug_name = request.form['drug_name']
        cell_line_name = request.form['cell_line_name']

        # Preprocess the input data and make predictions
        prediction = preprocess_input_gdsc(drug_name, cell_line_name)
        return render_template('index.html', prediction=prediction)
    return render_template('index.html')


@app.route('/precily', methods=['POST', 'GET'])
def predict_precily():
    if request.method == "POST":
        query = request.get_json()
        drug_name = query['drug_name']
        cell_line_name = query['cell_line_name']
        prediction = preprocess_input_prc(drug_name, cell_line_name)
        results = {'prediction': str(prediction)}
        return jsonify(results)

@app.route('/tcnn', methods=['POST', 'GET'])
def predict_tcnn_ic50():
    if request.method == "POST":
        query = request.get_json()
        drug_name = query['drug_name']
        cell_line_name = query['cell_line_name']
        raw_prediction = preprocess_input_tcnn(drug_name, cell_line_name)
        prediction = calculate_y(raw_prediction)
        results = {'prediction': prediction}
        return jsonify(results)

@app.route('/gdsc', methods=['POST', 'GET'])
def predict_gdsc():
    if request.method == "POST":
        query = request.get_json()
        drug_name = query['drug_name']
        cell_line_name = query['cell_line_name']
        prediction = preprocess_input_gdsc(drug_name, cell_line_name)
        results = {'prediction': str(prediction)}
        return jsonify(results)

In [14]:
drug_name = '5-Fluorouracil'
cell_line_name = '22RV1'

In [15]:
prediction = preprocess_input_gdsc(drug_name, cell_line_name)
prediction

GDSC Dict: dict_keys([('22RV1', '5-Fluorouracil'), ('22RV1', 'Acetalax'), ('22RV1', 'Afatinib'), ('22RV1', 'Afuresertib'), ('22RV1', 'AGI-5198'), ('22RV1', 'AGI-6780'), ('22RV1', 'Alisertib'), ('22RV1', 'Alpelisib'), ('22RV1', 'AMG-319'), ('22RV1', 'AT13148'), ('22RV1', 'AZ6102'), ('22RV1', 'AZ960'), ('22RV1', 'AZD1208'), ('22RV1', 'AZD1332'), ('22RV1', 'AZD2014'), ('22RV1', 'AZD3759'), ('22RV1', 'AZD4547'), ('22RV1', 'AZD5153'), ('22RV1', 'AZD5363'), ('22RV1', 'AZD5438'), ('22RV1', 'AZD5991'), ('22RV1', 'AZD6738'), ('22RV1', 'AZD7762'), ('22RV1', 'AZD8186'), ('22RV1', 'BIBR-1532'), ('22RV1', 'BMS-536924'), ('22RV1', 'Buparlisib'), ('22RV1', 'Camptothecin'), ('22RV1', 'Cediranib'), ('22RV1', 'Crizotinib'), ('22RV1', 'Cytarabine'), ('22RV1', 'CZC24832'), ('22RV1', 'Dabrafenib'), ('22RV1', 'Dactolisib'), ('22RV1', 'Daporinad'), ('22RV1', 'Dasatinib'), ('22RV1', 'Dihydrorotenone'), ('22RV1', 'Dinaciclib'), ('22RV1', 'Elephantin'), ('22RV1', 'Entinostat'), ('22RV1', 'Entospletinib'), ('22R

2.478859

In [10]:

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, use_reloader=True, port=3000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3000
 * Running on http://192.168.57.203:3000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/home/neeraj/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/neeraj/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/neeraj/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/neeraj/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 654, in initialize
    self.init_sockets()
  File "/home/neeraj/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/home/neeraj/.local/lib/python3.8/site-packages/ipykernel/ke

SystemExit: 1